<a href="https://colab.research.google.com/github/youssef2620/ASDC/blob/main/oxford%20flower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importing Needed Libraries**

In [1]:
import os
import requests
import tarfile
import scipy
from scipy import io
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image

In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

DataPath = '/content/drive/My Drive/data_final'
LabelsPath = os.path.join(DataPath, 'labels.csv')
if not os.path.exists(DataPath):
  os.mkdir(DataPath)


Mounted at /content/drive


## **Downloading Dataset**

In [3]:
tgz_path = os.path.join(DataPath, '102flowers.tgz')
url = 'http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz'
r_image = requests.get(url)
with open(tgz_path, 'wb') as f:
  f.write(r_image.content)
tar = tarfile.open(tgz_path, 'r')
for item in tar:
  tar.extract(item, DataPath)

In [4]:
mat_path = os.path.join(DataPath, 'imagelabels.mat')
label_url = 'http://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat'
r_label = requests.get(label_url)
with open(mat_path, 'wb') as f:
  f.write(r_label.content)

## **Data Integration and Labeling**

In [5]:
# Load labels and create DataFrame
matdata = scipy.io.loadmat(mat_path)
labels = matdata['labels'][0]
images = ['image_{:05}.jpg'.format(i + 1) for i in range(len(labels))]
image_label_df = pd.DataFrame({'image': images, 'label': labels})

actual_labels = ["pink primrose", "hard-leaved pocket orchid", "canterbury bells", "sweet pea", "english marigold", "tiger lily", "moon orchid",
        "bird of paradise", "monkshood", "globe thistle", "snapdragon", "colt's foot", "king protea", "spear thistle", "yellow iris",
        "globe-flower", "purple coneflower", "peruvian lily", "balloon flower", "giant white arum lily", "fire lily", "pincushion flower", "fritillary",
        "red ginger", "grape hyacinth", "corn poppy", "prince of wales feathers", "stemless gentian", "artichoke", "sweet william", "carnation",
        "garden phlox", "love in the mist", "mexican aster", "alpine sea holly", "ruby-lipped cattleya", "cape flower", "great masterwort", "siam tulip",
        "lenten rose", "barbeton daisy", "daffodil", "sword lily", "poinsettia", "bolero deep blue", "wallflower", "marigold", "buttercup", "oxeye daisy",
        "common dandelion", "petunia", "wild pansy", "primula", "sunflower", "pelargonium", "bishop of llandaff", "gaura", "geranium", "orange dahlia",
        "pink-yellow dahlia?", "cautleya spicata", "japanese anemone", "black-eyed susan", "silverbush", "californian poppy", "osteospermum",
        "spring crocus", "bearded iris", "windflower", "tree poppy", "gazania", "azalea", "water lily", "rose", "thorn apple", "morning glory",
        "passion flower", "lotus", "toad lily", "anthurium", "frangipani", "clematis", "hibiscus", "columbine", "desert-rose", "tree mallow",
        "magnolia", "cyclamen", "watercress", "canna lily", "hippeastrum", "bee balm", "ball moss", "foxglove", "bougainvillea", "camellia", "mallow",
        "mexican petunia", "bromelia", "blanket flower", "trumpet creeper", "blackberry lily"]



# Add actual labels to the DataFrame
image_label_df['actual_label'] = [actual_labels[label - 1] for label in labels]

# Create a new DataFrame with the specified format
result_df = pd.DataFrame({
    'image': image_label_df['image'],
    'label': image_label_df['label'],
    'actual_label': image_label_df['actual_label']
})

# Save the new DataFrame as a CSV file
csv_path = os.path.join(DataPath, 'image_label_name.csv')
result_df.to_csv(csv_path, index=False)


## **Splitting Dataset**

In [6]:
X_train_path = os.path.join(DataPath, 'X_train')
X_test_path = os.path.join(DataPath, 'X_test')
jpg_path = os.path.join(DataPath, 'jpg')
csv_path = os.path.join(DataPath, 'image_label_name.csv')

In [7]:
from shutil import copyfile

# Load labels from CSV
labels = pd.read_csv(csv_path)

# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(labels['image'], labels['actual_label'], test_size=0.2, random_state=0)

# Create directories if they don't exist
if not os.path.exists(X_train_path):
    os.mkdir(X_train_path)
if not os.path.exists(X_test_path):
    os.mkdir(X_test_path)

# Copy images to the corresponding directories
for f in os.listdir(jpg_path):
    img_path = os.path.join(jpg_path, f)
    img = Image.open(img_path)
    if f in X_train.values:
        img.save(os.path.join(X_train_path, f))
    elif f in X_test.values:
        img.save(os.path.join(X_test_path, f))


In [8]:
# Loop through images in the training set
for f in os.listdir(X_train_path):
    # Find the index of the image in the DataFrame
    index = labels.index[labels['image'] == f].tolist()[0]

    # Get the category based on the actual label
    category = str(labels.at[index, 'actual_label']).replace('[', '').replace(']', '').replace("'", '')

    # Handle special cases
    if category == "colts foot":
        category = "colt's foot"

    # Create category path if it doesn't exist
    category_path = os.path.join(X_train_path, category)
    if not os.path.exists(category_path):
        os.makedirs(category_path)

    # Move the image to the corresponding category directory
    img_path = os.path.join(X_train_path, f)
    img = Image.open(img_path)
    img.save(os.path.join(category_path, f))

    # Remove the original image
    os.remove(img_path)

In [9]:
# Loop through images in the testing set
for f in os.listdir(X_test_path):
    # Find the index of the image in the DataFrame
    index = labels.index[labels['image'] == f].tolist()[0]

    # Get the category based on the actual label
    category = str(labels.at[index, 'actual_label']).replace('[', '').replace(']', '').replace("'", '')

    # Handle special cases
    if category == "colts foot":
        category = "colt's foot"

    # Create category path if it doesn't exist
    category_path = os.path.join(X_test_path, category)
    if not os.path.exists(category_path):
        os.makedirs(category_path)

    # Move the image to the corresponding category directory
    img_path = os.path.join(X_test_path, f)
    img = Image.open(img_path)
    img.save(os.path.join(category_path, f))

    # Remove the original image
    os.remove(img_path)

## **Importing Needed Libraries for Training**

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


In [11]:
num_classes = 102
batch_size = 64
img_size = 224
epochs = 30
num_train = 6551
num_val = 1638
label = ["pink primrose", "hard-leaved pocket orchid", "canterbury bells", "sweet pea", "english marigold", "tiger lily", "moon orchid",
        "bird of paradise", "monkshood", "globe thistle", "snapdragon", "colt's foot", "king protea", "spear thistle", "yellow iris",
        "globe-flower", "purple coneflower", "peruvian lily", "balloon flower", "giant white arum lily", "fire lily", "pincushion flower", "fritillary",
        "red ginger", "grape hyacinth", "corn poppy", "prince of wales feathers", "stemless gentian", "artichoke", "sweet william", "carnation",
        "garden phlox", "love in the mist", "mexican aster", "alpine sea holly", "ruby-lipped cattleya", "cape flower", "great masterwort", "siam tulip",
        "lenten rose", "barbeton daisy", "daffodil", "sword lily", "poinsettia", "bolero deep blue", "wallflower", "marigold", "buttercup", "oxeye daisy",
        "common dandelion", "petunia", "wild pansy", "primula", "sunflower", "pelargonium", "bishop of llandaff", "gaura", "geranium", "orange dahlia",
        "pink-yellow dahlia?", "cautleya spicata", "japanese anemone", "black-eyed susan", "silverbush", "californian poppy", "osteospermum",
        "spring crocus", "bearded iris", "windflower", "tree poppy", "gazania", "azalea", "water lily", "rose", "thorn apple", "morning glory",
        "passion flower", "lotus", "toad lily", "anthurium", "frangipani", "clematis", "hibiscus", "columbine", "desert-rose", "tree mallow",
        "magnolia", "cyclamen", "watercress", "canna lily", "hippeastrum", "bee balm", "ball moss", "foxglove", "bougainvillea", "camellia", "mallow",
        "mexican petunia", "bromelia", "blanket flower", "trumpet creeper", "blackberry lily"]

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=45,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.5,
    shear_range=0.2
    )

val_datagen = ImageDataGenerator(rescale=1.0/255)

train_gen = train_datagen.flow_from_directory(
    X_train_path,
    target_size=(img_size, img_size),
    color_mode='rgb',
    batch_size=batch_size,
#     classes=label,
    class_mode='categorical',
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    X_test_path,
    target_size=(img_size, img_size),
    color_mode='rgb',
    batch_size=batch_size,
#     classes=label,
    class_mode='categorical',
    shuffle=True
)

Found 6551 images belonging to 102 classes.
Found 1638 images belonging to 102 classes.


# **Training using Transfer Learning**

In [13]:
from tensorflow.keras.applications import MobileNetV2

# Specify the input shape and create the MobileNetV2 base model
img_size = 224  # Adjust the image size based on your requirements
base_model = MobileNetV2(input_shape=(img_size, img_size, 3), alpha=0.5, include_top=False, weights='imagenet')

# Optional: Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(102, activation='softmax')(x)  #102 classes for classification

# Create the final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()


3201480/3201480 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 16)         432       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 16)         64        ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (N

In [14]:
history = model.fit(
    train_gen,
    steps_per_epoch=num_train//batch_size,
    epochs=60,
    validation_data=val_gen,
    validation_steps=num_val//batch_size
)

Epoch 1/60
102/102 [==============================] - 127s 1s/step - loss: 2.9290 - accuracy: 0.3697 - val_loss: 1.6852 - val_accuracy: 0.6425
Epoch 2/60
102/102 [==============================] - 118s 1s/step - loss: 1.3838 - accuracy: 0.6986 - val_loss: 1.0884 - val_accuracy: 0.7725
Epoch 3/60
102/102 [==============================] - 121s 1s/step - loss: 1.0054 - accuracy: 0.7728 - val_loss: 0.8466 - val_accuracy: 0.8188
Epoch 4/60
102/102 [==============================] - 123s 1s/step - loss: 0.8219 - accuracy: 0.8064 - val_loss: 0.7357 - val_accuracy: 0.8263
Epoch 5/60
102/102 [==============================] - 120s 1s/step - loss: 0.7038 - accuracy: 0.8366 - val_loss: 0.6689 - val_accuracy: 0.8350
Epoch 6/60
102/102 [==============================] - 123s 1s/step - loss: 0.6541 - accuracy: 0.8421 - val_loss: 0.6097 - val_accuracy: 0.8506
Epoch 7/60
102/102 [==============================] - 121s 1s/step - loss: 0.5811 - accuracy: 0.8582 - val_loss: 0.5735 - val_accuracy: 0.8556

In [15]:
model.save('oxflower.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
